# Ejercicio de la Práctica 5: Automatic Differentation (AD) in Pytorch


En esta ejercicio nos centraremos en la implementación de AD en Pytorch. En concreto, en el submódulo [TORCH.AUTOGRAD](https://pytorch.org/docs/stable/autograd.html) 

Si no tienes instalado Pytorch, lo primero que has de hacer es instalarlo. Una forma sencilla de hacerlo es en el **Powershell Prompt** de **Anaconda**, escribiendo:

**conda install pytorch torchvision torchaudio pytorch-cuda=11.6 -c pytorch -c nvidia**

##  AD en Pytorch

Carga Pytorch escribiendo 

**import torch**

In [1]:
# Completar aquí
import torch
import numpy as np
from torch.autograd.functional import hessian
from numpy.linalg import eig
from torch.autograd.functional import hessian

# --------------------


El objetivo de este ejercicio es hacer los mismos cálculos que hicimos en la práctica de AD en TensorFlow pero ahora en Pytorch

Veamos un primer ejemplo elemental: derivada de la función $y = x^2$ en $x=3$.

In [2]:
# Completar aquí
x = torch.tensor([3.0], requires_grad=True)

y = x ** 2

# Calcula la derivada de y respecto a x
y.backward()

# El gradiente de y respecto a x está en x.grad
dy_dx = x.grad
print(f"derivada de y respecto a x = {dy_dx}")

# --------------------


derivada de y respecto a x = tensor([6.])


Consideremos ahora una función de dos variables

$$
f(x_1, x_2) = x_1^2(x_1 + x_2)
$$

Vamos a calcular su gradiente en $x_1 = 2$, $x_2 = 3$. 

In [3]:
# Completar aquí
# tensor que requiere gradientes
x = torch.tensor([2.0, 3.0], requires_grad=True)

f = x[0]**2 * (x[0] + x[1])

# Calculamos el gradiente de f con respecto a x
f.backward()  # Esto calcula el gradiente

# Accede al gradiente de x
grad_f = x.grad
print(f"gradiente de f respecto a x = {grad_f}")



gradiente de f respecto a x = tensor([24.,  4.])


Calculamos ahora su matriz Hessiana en el mismo punto y los valores propios de dicha matriz

In [4]:
def f(x):
    return x[0]**2 * (x[0] + x[1])

x = torch.tensor([2.0, 3.0], requires_grad=True)

hess_f = hessian(f, x)
print("Hessian matrix:\n", hess_f)


eigenvalues, _ = eig(hess_f)

print(f"valores propios de la matriz hessiana de f en (2, 3) = \n {eigenvalues}")


Hessian matrix:
 tensor([[18.,  4.],
        [ 4.,  0.]])
valores propios de la matriz hessiana de f en (2, 3) = 
 [18.848858  -0.8488578]


Veamos ahora un modelo más elaborado. Consideremos la función vectorial 
$$
y = x * w  + b
$$
donde $x$ es un vector fila de $4$ componentes, $w$ es una matriz $4\times 3$ y $b$ un vector columna de $3$ componentes.

Define la función anterior en **pytorch** y asigna los siguientes valores a la variables:

1)  $x = [[1., 2., 3., 4.]]$
2)  $w$ valores aleatorios
3)  $b$ unos

In [5]:
# Completar aquí
tensor_x = torch.tensor([[1, 2, 3, 4]], dtype=torch.float32, requires_grad=True)
tensor_w = torch.rand((4, 3), dtype=torch.float32)  # w: peso
tensor_b = torch.ones(3, dtype=torch.float32)  # bias

tensor_y = tensor_x @ tensor_w + tensor_b

print("x = ", tensor_x)
print("w =\n", tensor_w)
print("b = tensor", tensor_b)
print("tensor_y:", tensor_y)

# --------------------


x =  tensor([[1., 2., 3., 4.]], requires_grad=True)
w =
 tensor([[0.4792, 0.8692, 0.3657],
        [0.2091, 0.2768, 0.3109],
        [0.7517, 0.5404, 0.0791],
        [0.6101, 0.5677, 0.1531]])
b = tensor tensor([1., 1., 1.])
tensor_y: tensor([[6.5929, 6.3148, 2.8371]], grad_fn=<AddBackward0>)


Además de la función $y = x * w + b$ consideramos la función de pérdida

$$
\text{loss } = \frac{1}{3} \sum_{j=1}^3 (y_j - (y_{\text{label}})_j)^2
$$
donde $y_{\text{label}} = [[1., 2., 3.]]$

Define la función loss en pytorch

In [6]:
# Completar aqu
y_label = torch.tensor([[1, 2, 3]], dtype=torch.float32)

# torch.no_grad evita que se calcule el gradiente en los parámetros de la pérdida
with torch.no_grad():
    # Calcula la salida
    tensor_y = tensor_x @ tensor_w + tensor_b  # tensor_y = x @ w + b
    # Calcula la pérdida
    tensor_loss = torch.mean((tensor_y - y_label) ** 2)  # Pérdida MSE

print('loss: ', tensor_loss)

# --------------------


loss:  tensor(16.6414)


Calcula en pytorch los gradientes de la función loss, primero respecto a $y$ y después respecto a $x$

In [7]:
# Completar aquí
tensor_y = torch.matmul(tensor_x, tensor_w) + tensor_b  # tensor_y = x @ w + b
tensor_loss = torch.mean((tensor_y - y_label) ** 2)  # Pérdida

# Gradiente de la pérdida respecto a y
tensor_dloss_dy = torch.autograd.grad(tensor_loss, tensor_y, retain_graph=True)[0]

# Gradiente de la pérdida respecto a x
tensor_dloss_dx = torch.autograd.grad(tensor_loss, tensor_x)[0]

print('gradiente de loss respecto a y: ', tensor_dloss_dy)
print('gradiente de loss respecto a x: ', tensor_dloss_dx)
# --------------------


'''
Recordar:
Es necesario usar el parámetro "retain_graph=True" porque PyTorch trabaja con gráficos computacionales.
Cuando se calcula un gradiente con "torch.autograd", el gráfico se libera automáticamente para ahorrar memoria, 
lo que impide calcular gradientes múltiples en el mismo gráfico y daría error.
El parámetro en "True", le indica que mantenga el gráfico en memoria, permitiendo 
realizar varios calculos con el mismo gráfico computacional. 
'''


gradiente de loss respecto a y:  tensor([[ 3.7286,  2.8765, -0.1086]])
gradiente de loss respecto a x:  tensor([[4.2473, 1.5420, 4.3485, 3.8913]])


'\nRecordar:\nEs necesario usar el parámetro "retain_graph=True" porque PyTorch trabaja con gráficos computacionales.\nCuando se calcula un gradiente con "torch.autograd", el gráfico se libera automáticamente para ahorrar memoria, \nlo que impide calcular gradientes múltiples en el mismo gráfico y daría error.\nEl parámetro en "True", le indica que mantenga el gráfico en memoria, permitiendo \nrealizar varios calculos con el mismo gráfico computacional. \n'